# Automatic differentiation graph


## Introduction

Up to this point we used ready made algorithms from `sklearn` except for linear regression with analytical solution. When we called `fit` the algorithm was trained on our data.

All machine learning algorithms are essentially functions taking some input and producing some output and could be written like this:

$$
\hat{Y} = f(X)
$$

where `X` is data and $\hat{Y}$ is prediction. Those functions depend on `parameters` or `weights` (denoted by $\theta$) so our mathematical description changes to:

$$
\hat{Y} = f_{\theta}(X)
$$

Also there is `loss` which measures the error (always positive scalar value) compared to true values (`Y`). The less, the better as we saw previously. In maths:

$$
L(f_{\theta}(X), Y)
$$

Some of those functions (including neural networks) are differentiable, hence we can __calculate partial derivatives__.

Automatic differentiation graph allows us to compute `gradient` of `loss` ($\nabla L$) with respect to (abbreviated often as w.r.t.) parameters $\theta$ using chain rule, which one can write like this:

$$
\nabla L_{\theta}
$$

In this section we will only focus on `backpropagation` itself, loss and how it fits within Machine Learning domain will be shown in the next lesson.

## Intuition

Intuitively, we can think of parameters gradient as:

> how and by how much we should change their value in order to improve loss (decrease it)

## What is graph?

> Graph is a data structure containing __nodes__ and __edges__ connecting one node to another

Example image of graph will tell you everything you need to know:

![](images/graph_data_structure.png)

We can see you can travel across the nodes by moving along the edges, here are some questions:
- What is the path to get to get from A to E?
- Which path has the __largest__ and __smallest__ cost for A to E traversal?
- What are the __cycles__ inside this graph?
- What are the nodes which one __cannot reach__ from A and which from E?

## Undirected vs directed graph

Graphs can either __directed__ or __undirected__:

### Undirected graph

> Undirected graph can be traversed along edges __in any direction__

![](images/undirected_graph.png)

### Directed graph

> Directed graph can be traversed along edges in a __specified direction__

We have seen an example at the very top

### Directed acyclic graph

> Directed acyclic graph (DAG) can be traversed in a __specified direction__ and __has no cycles__

![](images/dag.png)

__This data structure is especially important for us!__

It will be used as a structure which will keep our automatic differentiation graph and is used across machine learning all the time (especially in deep learning and neural networks)

## Graph and chain rule

Let's write an example chain of functions:

$$
a = b(c(d(e(X))))
$$

Now, we would like to know how change in `e(X)` influences `a` value. To do that, we have to calculate gradient of `a` w.r.t. to `e`. Using chain rule this would be (each derivative is evaluated at certain point starting with `X`, left it out for brevity):

$$
\frac{da}{de} = \frac{da}{db} \frac{db}{dc} \frac{dc}{dd} \frac{dd}{de}
$$

> backpropagation is an algorithm which given output (`a` in this case) runs operations (__their derivative formulas__) backward in order to calculate gradient of inputs

`backprop` in computer programming can be described as graph, conceptually:
- when the value is calculated (forward pass) graph records each __operation__ on data
- when special function is run on the graph (we will name it `backward`), __derivatives__ of recorded operations are run in opposite order
- when we get to the __parameter__ we update it's gradient (in example above `e` would be a parameter)

__Let's see in visual form how the graph looks like:__

![](images/comp_graph.jpg)

## Dynamic vs static graph

There are out-of-the box frameworks which provide highly optimized graph implementations.
Two major approaches exist:

- static graph
- dynamic graph

They are mostly used in __neural networks__ (which we are gonna see in a separate module)

### Static graph

> graph is defined only once and cannot be updated afterwards

Best known representant of this approach in industry is [Tensorflow](https://www.tensorflow.org/), especially with it's `1.x` version. Currently running out of favours among developers (and researchers especially) and is a good fit for specific usage (very large neural networks).

#### Upsides

- Easier to distribute graph among multiple machines. Structure is known and can be shared more easily
- Space for optimization (though it's impact is highly debatable)
- Hence __might__ run faster

#### Downsides

- Does not feel Pythonic and is hard to use
- Cannot be changed based on any condition, only based on part of the framework
- Cumbersome syntax and usage; `if`, `for` and other constructs from Python do not exist
- Hard to debug

### Dynamic graph

> graph is defined "on the fly" and nodes are added as they are run

Best known representant of this approach is [PyTorch](https://pytorch.org/), while [Tensorflow](https://www.tensorflow.org/) tries to go the same path since `2.x` version.
Largely favored among researchers with weaker industry adoption (though that changes rapidly).

#### Upsides

- Easy to use. If implemented well can feel just like writing in Python and `numpy`
- Easier to debug
- Can be changed however we wish based on any condition

#### Downsides

- Graph is harder to distribute as it might change any time. Sharing it might be costly
- Might be harder to optimize (though optimized versions exist and usually isn't a problem)
- __Might__ run slower

As the last one is getting more and more traction we will make our own simplistic version of dynamic graph.

## Graph implementation

Let's start with `graph` and how it works. Conceptually, it will need the following:
- Keeping `parameters` and `operations` references inside
- Functions to register `parameters` and `operations` inside graph
- `backward` function to populate parameters with gradient

Below is a `class` which does it. Also as it is a dynamic graph `operations` have to be cleared after each `backward` call so they can be redefined "on-the-fly".

__Don't worry, we will walk you through it step by step!__

In [ ]:
import contextlib


class Graph:
    """Graph class used for backward automatic differentiation (backpropagation).


    Can only differentiate w.r.t. scalar values. `backpropagate` function
    should be called on final parameter (after all operations
    were performed).

    Attributes:
        operations (Dict[int, (Operation, Dict[int, (int, bool)])]):
            List of operations which, when backpropagated produce gradients
            for Parameters. Each item is a Tuple containing:
            - Instance of operation
            - Dictionary containing:
                - index of input parameter (so usually it is [0, 1, 2, 3...])
                - Tuple containing:
                    - index of operation which created this input parameter
                    - True/False value whether this node is a leaf

            If node is a leaf it has to be parameter and backpropagation stops
            at this call to `backward` (see `Parameter` class)

        parameters (List[Parameter])
            List of parameters added to this graph.
    """

    def __init__(self):
        self.operations = {}
        self.parameters = []

    def _register_parameter(self, parameter: "Parameter"):
        """Registers parameter inside the graph

        Arguments:
            Instance of Parameter to be registered

        Returns:
            Index of parameter inside the graph which is saved in parameter's instance.

        """
        self.parameters.append(parameter)
        return len(self.parameters) - 1

    def _register_operation(self, operation: "Operation", inputs):
        """Registers operation inside the graph

        Returns:
            Index of operation inside the graph which is saved in parameter's instance.

        """
        if has_grad():
            last_index = len(list(self.operations.keys()))
            self.operations[last_index] = (operation, inputs)
            return last_index

    @staticmethod
    def _get_gradient(upstream_gradient, output_index):
        """If gradient is a Tuple return element otherwise return upstream_gradient
        as is.

        """
        if isinstance(upstream_gradient, (tuple, list)):
            return upstream_gradient[output_index]
        return upstream_gradient

    def _backpropagate_node(
        self, upstream_gradient, output_index, operation_index, is_leaf
    ) -> None:
        """Backpropagate through single node (Operation or Parameter).

        If Parameter (is_leaf=True) is reached backpropagation will end with
        populating it's gradient.

        If Operation (is_leaf=False) is reached the node will be run through
        graph backpropagation again.

        """
        gradient = Graph._get_gradient(upstream_gradient, output_index)
        if is_leaf:
            self.parameters[operation_index].backward(gradient)
        else:
            # If we went through this operation we should raise an error
            new_operation = self.operations.pop(operation_index, None)
            if new_operation is None:
                raise ValueError(
                    "Trying to backpropagate through non-existent node. Are your paths disjoint?"
                )
            self._backpropagate_graph(new_operation, gradient)

    def _backpropagate_graph(self, operation_and_mapping, upstream_gradient):
        """Backpropagate through graph of operations.

        For any incoming operation it will go over their inputs
        (defined by mapping which points to input nodes) and propagate
        current upstream gradient to them.

        After calculation of gradient it's internal cached is clear by the graph

        """
        operation, mapping = operation_and_mapping
        upstream_gradient = operation.backward(upstream_gradient)
        # Clean cache
        operation.cache = None
        # Multiple outputs
        for output_index, (operation_index, is_leaf) in mapping.items():
            self._backpropagate_node(
                upstream_gradient, output_index, operation_index, is_leaf
            )

    def backward(self, upstream_gradient=1) -> None:
        """Entrypoint for backpropagation through registered nodes.

        `backward` will run through all the nodes contained inside graph in succession,
        starting with the one added as the last one.

        If there are multiple __separable__ paths those will be backpropagated
        as well. If they aren't separable an error will be raised.

        When graph's `backward` is called it will be cleaned from
        all operations (parameters stay inside graph until the graph instance
        is available, usually throughout the whole program).

        """
        if not has_grad():
            raise ValueError("Cannot perform backward as tape recording is off.")

        while self.operations:
            last_index = list(self.operations.keys())[-1]
            self._backpropagate_graph(
                self.operations.pop(last_index), upstream_gradient
            )
        for parameter in self.parameters:
            parameter.last_operation_index = None


class _GlobalGraph:
    "Class used to hide global state from the main namespace."
    # even though this stuff is inside this class definition, it gets run!
    graph = Graph() # accessed as _GlobalGraph.graph
    on: bool = True # accessed as _GlobalGraph.on
    print('Created graph')


def get():
    """Return global graph"""
    return _GlobalGraph.graph


def has_grad():
    return _GlobalGraph.on


@contextlib.contextmanager # the following function can be used in "with no_grad() as x:"
def no_grad():
    """
    Creates context for perfoming graph operations without doing everyhing required for backpropagation (like storing gradients)

    Usage: 
        with no_grad():
            # at this point everthing before the yield statement happens

            # do things without storing grads etc
            # e.g. make predictions in production
            # makes things happen as fast as possible
            # makes things that not take up extra memory

        # here (once i've exited the context i.e. unindented again) the stuff after the yield statement happens

    """
    _GlobalGraph.on = False
    yield
    _GlobalGraph.on = True



Please notice that __graph is global__. It is simpler that way as `parameters` are registered only once.

## Exercise

__Why initial gradient is 1?__

### Solution

Simply, we take derivative of the last value w.r.t to itself. Let's call this value `x`:

$$
\frac{dx}{dx} = 1
$$

And as we know derivative of `x` is simply `1`, hence that is the value we feed the graph.

## Operation

Next let's create `Operation`. Let's recap what is needed:
- take data value and calculate mathematical operation on it
- register itself in graph so it can be run during `backward`
- implement `backward`

This is exactly what `base classes` are for. Later we can implement specific math operation by deriving from this class:

In [ ]:
import abc

class Operation(abc.ABC):
    """Base class of mathematical operation to be run on Parameter/np.array

    Attributes:
        cache (Optional[np.array])
            Cache attribute one can use to save anything during forward pass
            to reuse in backward
        index_in_graph (int):
            Index of operation in graph's operation dictionary
        is_leaf (bool):
            Always `False`, used by graph to easily discern between parameters
            and operations.
    """

    def __init__(self):
        self.cache = None
        self.index_in_graph = None
        self.is_leaf = False

    def __call__(self, *arguments):
        """Run forward and register operation in graph.

        Additionally operation's inputs will be registered using mapping and
        whether those are leafs (parameters) or operations to be further
        backpropagated.

        """
        if has_grad():
            mapping = {}
            add_to_graph = False
            for input_index, argument in enumerate(arguments):
                if isinstance(argument, Parameter):
                    add_to_graph = True
                    is_first_operation = argument.last_operation_index is None
                    if is_first_operation:
                        mapping[input_index] = (argument.index_in_graph, True)
                    else:
                        mapping[input_index] = (argument.last_operation_index, False)

            if add_to_graph:
                self.index_in_graph = get()._register_operation(self, mapping)
                for argument in arguments:
                    if isinstance(argument, Parameter):
                        argument.last_operation_index = self.index_in_graph

        # Pack return value in tuple always
        return self.forward(*arguments)

    @abc.abstractmethod
    def forward(self, *_):
        """Define your forward pass here.

        Use self.cache to cache anything needed during backpropagation.

        """
        pass

    @abc.abstractmethod
    def backward(self, upstream_gradient):
        """Define your backward pass here.

        Use self.cache in order to calculate gradient. There has to be as
        many outputs as there was inputs to forward.

        """
        pass

## Parameter

Last class we need is `Parameter`. As we know `np.array` and how it works we are going to base our implementation by inheriting from this class.

`numpy.ndarray` requires special approach when inheriting from it. `__new__` instead of `__init__` and `__array_finalize__` special method. For those curious [you can read more about it here](https://numpy.org/doc/stable/user/basics.subclassing.html).

What we need for our `Parameter`?
- `gradient` attribute which keeps the gradient (or `None` if `backward` wasn't called)
- when `backward` is called, `self.gradient` should be populated
- `gradient` should be cleared when it isn't needed anymore. `clear` method will simply assign `None` to it

In [ ]:
import itertools
import numpy as np

# https://numpy.org/doc/stable/user/basics.subclassing.html
class Parameter(np.ndarray):
    """Parameter class to be populated with gradient.

    Attributes:
        gradient (Optional[np.array]):
            Array with gradients with which parameter can be optimized via
            optimizer
        index_in_graph (int):
            Index of parameter in graph's list
        is_leaf (bool):
            Always True, used by graph to easily discern between parameters
            and operations.
    """

    def __new__(cls, input_array):
        # Input array is an already formed ndarray instance
        # We first cast to be our class type
        obj = np.asarray(input_array).view(cls)
        # Gradient is None until populated
        obj.gradient = None
        obj.index_in_graph = get()._register_parameter(obj)
        obj.is_leaf = True
        obj.last_operation_index = None
        # Return newly created object
        return obj

    # Don't sweat over it, just assigning the same attributes as in new
    def __array_finalize__(self, obj):
        """Re-assign data contained in parameter.

        Workaround for `numpy` subclassing.

        """
        if obj is None:
            return
        self.gradient = getattr(obj, "gradient", None)
        self.index_in_graph = getattr(obj, "index_in_graph", None)
        self.is_leaf = getattr(obj, "is_leaf", True)
        self.last_operation_index = getattr(obj, "last_operation_index", None)

    def broadcast_fix(self, gradient):
        """Try to fix numpy's broadcasting with gradient.

        `1` dimensions may be broadcasted to other automatically. There is no
        clear way to know about that which could be easily implemented.

        Broadcasting is equal to summing all the values, hence __any__ dimension
        which might be off in gradient is summed by the function below.

        """
        if not isinstance(gradient, np.ndarray):
            return gradient

        if gradient.flatten().shape == self.flatten().shape:
            return gradient
        flattened_gradient = gradient.flatten()
        flattened_data = self.flatten()
        if len(flattened_gradient.shape) < len(flattened_data.shape):
            raise ValueError(
                "Data has more dimension than gradient, something went very wrong."
            )

        to_sum = []
        # Gradient cannot be None as the condition is checked above
        for index, (data_shape, gradient_shape) in enumerate(
            itertools.zip_longest(flattened_data.shape, flattened_gradient.shape)
        ):
            if data_shape is None or gradient_shape > data_shape:
                to_sum.append(index)
            if data_shape > gradient_shape:
                raise ValueError("Data has more elements than it's gradient")
        return np.sum(flattened_gradient, axis=tuple(to_sum)).flatten()

    def backward(self, upstream_gradient) -> None:
        """Take upstream gradient and update param's gradient with it."""
        self.gradient = self.broadcast_fix(upstream_gradient)

    def clear(self) -> None:
        """Clear gradient to save RAM memory."""
        self.gradient = None

## Implement operations

Now that we have defined our classes we can implement concrete `operations`. Let's start easy with `addition` to see what this is all about.

## Addition

So all we have to implement, as seen previously, is `forward` and `backward`. We will also wrap the object in `add` function to use it easier afterwards.

## Exercise

Formula for addition is known to everyone:

$$
c = a + b
$$

`c` is influenced by `a` and `b`, hence we have to calculate gradient w.r.t. to both of them:

$$
\begin{align}
    \frac{\partial c}{da}(a + b) = 1 \\
    \frac{\partial c}{db}(a + b) = 1
\end{align}
$$

So now we have all we need, let's put this in code!

In [ ]:
# Concrete implementations
class _Add(Operation):
    # Simply add two values
    def forward(self, a, b):
        return a + b

    # Return gradient for a and b (remember)
    def backward(self, upstream_gradient):
        return upstream_gradient, upstream_gradient

# Create object and run it with (a, b)
def add(a, b):
    return _Add()(a, b)

## Mean

We will also implement second operation (for now, in the whole chapter we will implement a few more).

## Exercise

### Part 1

Formula for mean (denoted `m` here) of `N` variables is:

$$
m = \frac{1}{N}\sum_{i=1}^{N}x_i
$$

As we have `N` variables we will have to return `N` values. Fortunately, during implementation, as we take `mean` of `np.array` we will return `np.array` of the same shape.

Derivative this time will be a little harder, so let's so let's start small and calculate derivative of `1` variable:

$$
\frac{\partial m}{\partial x_1}(\frac{1}{1} * x_1) = 1
$$

Try the same with two variables, maybe `3`. What would the result be for `N` variables?

### Part 1 solution

Let's try it for two variables:

$$
\begin{align}
    \frac{\partial m}{\partial x_1}(\frac{1}{2} * (x_1 + x_2)) = \frac{1}{2} \\
    \frac{\partial m}{\partial x_2}(\frac{1}{2} * (x_1 + x_2)) = \frac{1}{2}
\end{align}
$$

Finally for three:

$$
\begin{align}
    \frac{\partial m}{\partial x_1}(\frac{1}{3} * (x_1 + x_2 + x_3)) = \frac{1}{3} \\
    \frac{\partial m}{\partial x_2}(\frac{1}{3} * (x_1 + x_2 + x_3)) = \frac{1}{3} \\
    \frac{\partial m}{\partial x_3}(\frac{1}{3} * (x_1 + x_2 + x_3)) = \frac{1}{3} \\
\end{align}
$$

So for `N` variables each will have `1/N` derivative. 

### Part 2

Implement `_Mean` operation and `mean` function in analogous way we did with `_Add` and `add`:

In [ ]:
class _Mean(Operation):
    def __init__(self, axis: int = None):
        super().__init__()
        self.axis = axis

    def forward(self, inputs):
        mean = np.mean(inputs, axis=self.axis)
        self.cache = np.ones_like(inputs) / inputs.size
        return mean

    def backward(self, upstream_gradient):
        return upstream_gradient * self.cache

def mean(inputs, axis: int = None):
    return _Mean(axis)(inputs)

## Running graph

As we have everything in place, let's use it. __Remember to use functions NOT CLASSES__, it should be pretty natural.

Your task is to create two `Parameters` from `np.random.randn` arrays of shape `10, 5`, __add__ them together, take the __mean__, backpropagate and check gradients.

In [ ]:
np.random.seed(0)

x1 = Parameter(np.random.randn(10, 5))
x2 = Parameter(np.random.randn(10, 5))

forward_result = mean(add(x1, x2))
get().backward()

x1.gradient, x2.gradient

## Challenges

- Learn more about graphs. You can start from [wikipedia](https://en.wikipedia.org/wiki/Graph_(abstract_data_type)) and move on to [graph traversal](https://en.wikipedia.org/wiki/Graph_traversal)
- Code your own simple graph and implement [BFS](https://en.wikipedia.org/wiki/Breadth-first_search) and [DFS](https://en.wikipedia.org/wiki/Depth-first_search) on it. What are the advantages/disadvantages of one method over the other?
- Go through this lesson multiple times. This one is hard so make sure you understand what is going on, ask questions if needed
- Read about [reverse (backward) vs forward automatic differentiation](https://math.stackexchange.com/questions/2195377/reverse-mode-differentiation-vs-forward-mode-differentiation-where-are-the-be). Which frameworks implement both (tip: check out [JAX](https://github.com/google/jax))?